# Brain Tumor Segmentation (BraTS) 2020 - NII to 3D NumPy Array

![CT Image](https://storage.googleapis.com/kaggle-datasets-images/674071/1185670/0d449dc88ac1318321ae8f7de974f0fa/dataset-cover.jpg?t=2020-05-25-13-21-33)

- **Author:** *Mariusz Wiśniewski*
- **Date created:** *April 17th, 2023*
- **Last modified:** *April 17th, 2023*

## Overview

In this notebook, we will convert volumes from the *BraTS2020* dataset from *.NII* to 3D NumPy arrays.

All BraTS multimodal scans are available as NIfTI files (.nii.gz) and describe:

- native (**T1**),
- post-contrast T1-weighted (**T1Gd**),
- T2-weighted (**T2**),
- T2 Fluid Attenuated Inversion Recovery (**T2-FLAIR**)

volumes, and were acquired with different clinical protocols and various scanners from multiple (n=19) institutions.

All the imaging datasets have been segmented manually, by one to four raters, following the same annotation protocol, and their annotations were approved by experienced neuro-radiologists. Annotations comprise:

- the necrotic and non-enhancing tumor core (**NCR/NET — label 1**),
- the peritumoral edema (**ED — label 2**),
- the GD-enhancing tumor (**ET — label 4**),
 
as described both in the [BraTS 2012-2013 TMI paper](https://ieeexplore.ieee.org/document/6975210) and in the [latest BraTS summarizing paper](https://arxiv.org/abs/1811.02629). The provided data are distributed after their pre-processing, i.e., co-registered to the same anatomical template, interpolated to the same resolution ($1 {mm}^3$) and skull-stripped.

## References

- [📄 The Multimodal Brain Tumor Image Segmentation Benchmark (BRATS)](https://ieeexplore.ieee.org/document/6975210)
- [📄 Advancing The Cancer Genome Atlas glioma MRI collections with expert segmentation labels and radiomic features](https://www.nature.com/articles/sdata2017117)
- [📄 Identifying the Best Machine Learning Algorithms for Brain Tumor Segmentation, Progression Assessment, and Overall Survival Prediction in the BRATS Challenge](https://arxiv.org/abs/1811.02629)

## Import Statements

In [1]:
import os
from glob import glob

import nibabel as nib
import numpy as np
import pandas as pd
from scipy import ndimage

## Project Configuration

In [2]:
DATA_PATH = './archive_BraTS2022'
TRAIN_PATH = f'{DATA_PATH}/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData'
TEST_PATH = f'{DATA_PATH}/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData'
DATA_TYPES = ['flair', 't1', 't1ce', 't2', 'seg']
N_FOLDS = 5

# Dataset

## Data Distribution

In [11]:
train_data_paths = {
    data_type: sorted(
        glob(f'{TRAIN_PATH}/**/*_{data_type}.nii')
    ) for data_type in DATA_TYPES
}
train_data_paths['seg'].append(f'{TRAIN_PATH}/BraTS20_Training_355/W39_1998.09.19_Seg.nii')
train_data_paths['seg'] = sorted(train_data_paths['seg'])

for k, v in train_data_paths.items():
    print(f'[TRAIN] Number of {k} images: {len(v)}')
print()
test_data_paths = {
    data_type: sorted(
        glob(f'{TEST_PATH}/**/*_{data_type}.nii')
    ) for data_type in DATA_TYPES
}

for k, v in test_data_paths.items():
    print(f'[TEST] Number of {k} images: {len(v)}')

[TRAIN] Number of flair images: 369
[TRAIN] Number of t1 images: 369
[TRAIN] Number of t1ce images: 369
[TRAIN] Number of t2 images: 369
[TRAIN] Number of seg images: 369

[TEST] Number of flair images: 125
[TEST] Number of t1 images: 125
[TEST] Number of t1ce images: 125
[TEST] Number of t2 images: 125
[TEST] Number of seg images: 0


## Loading Data and Preprocessing

In [12]:
def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan


# def normalize(volume):
#     """Normalize the volume"""
#     volume = (volume - np.min(volume)) / (np.max(volume) - np.min(volume))
#     return (volume * 255).astype(np.uint8)


def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(path)
    # Normalize
    # volume = normalize(volume)
    # Rotate by -90 degrees
    # volume = ndimage.rotate(volume, -90, axes=(0,1), reshape=False, order=1)

    return volume

In [13]:
file_path = '/Users/choihanjun/Library/Mobile Documents/com~apple~CloudDocs/Downloads/2024-1/[강화학습]/Project/archive_BraTS2022/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_seg.nii'

processed_volume = process_scan(file_path)
unique_values = np.unique(processed_volume)
print(unique_values)

[0. 1. 2. 4.]


# Visualization

In [20]:
object_list = []
# processed_volume.shape = (240, 240, 155)
dx = []
dy = []
dz = []

for x in range(-1, 2):  # -1, 0, 1
    for y in range(-1, 2):
        for z in range(-1, 2):
            if x == 0 and y == 0 and z == 0:
                continue  # 중심점 제외
            dx.append(x)
            dy.append(y)
            dz.append(z)

print(dx)
print(dy)
print(dz)            
    

[-1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[-1, -1, -1, 0, 0, 0, 1, 1, 1, -1, -1, -1, 0, 0, 1, 1, 1, -1, -1, -1, 0, 0, 0, 1, 1, 1]
[-1, 0, 1, -1, 0, 1, -1, 0, 1, -1, 0, 1, -1, 1, -1, 0, 1, -1, 0, 1, -1, 0, 1, -1, 0, 1]


In [28]:
from collections import deque



def make_bbox(file_path):
    file_path = '/Users/choihanjun/Library/Mobile Documents/com~apple~CloudDocs/Downloads/2024-1/[강화학습]/Project/archive_BraTS2022/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_seg.nii'
    processed_volume = process_scan(file_path)  
    visited = np.zeros(processed_volume.shape)
    len_z, len_y, len_x = processed_volume.shape
    
    def check(visited, nxt_z, nxt_y, nxt_x):
        # 밖으로 나가지 않았고, 방문하지 않았고, tumor이라면 nxt 방문 조건이 됨.
        if 0 <= nxt_z < visited.shape[0] and \
            0 <= nxt_y < visited.shape[1] and \
                0 <=  nxt_x < visited.shape[2] and \
                    visited[nxt_z][nxt_y][nxt_x] != 1 and \
                        processed_volume[nxt_z][nxt_y][nxt_x] == 4:
            return True
        else:
            return False
        
    def BFS(z, y, x):
        deq = deque([(z, y, x)])
        object_list = [(z, y, x)]
        while deq:
            now_pos = deq.popleft()
            for i in range(len(dx)):
                nxt_z, nxt_y, nxt_x = now_pos[0] + dz[i], now_pos[1] + dy[i], now_pos[2] + dx[i]
                if check(visited, nxt_z, nxt_y, nxt_x):
                    visited[nxt_z][nxt_y][nxt_x] = 1
                    deq.appendleft((nxt_z, nxt_y, nxt_x))
                    object_list.append((nxt_z, nxt_y, nxt_x))
                    
        if object_list:
            zs, ys, xs = zip(*object_list)
            min_z, max_z, min_y, max_y, min_x, max_x = min(zs), max(zs), min(ys), max(ys), min(xs), max(xs)
            return min_z, min_y, min_x, max_z, max_y, max_x
        else:
            return None
    
    
    for z in range(len_z):
        for y in range(len_y):
            for x in range(len_x):
                if check(visited, z, y, x):
                    bbox = BFS(z, y, x)
                    if bbox:
                        print(f"BBox from ({bbox[0]}, {bbox[1]}, {bbox[2]}) to ({bbox[3]}, {bbox[4]}, {bbox[5]})")
                    else:
                        print("No object found at this starting point")
                    
    



make_bbox(' ')

BBox from (66, 99, 39) to (119, 160, 82)
BBox from (75, 115, 53) to (75, 115, 53)
BBox from (112, 121, 50) to (112, 121, 50)
